In [1]:
# Import necessary libraries
import random
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model, Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
import os
from dotenv import load_dotenv

In [2]:
# COMMON VARIABLES
load_dotenv()
intents_name = os.getenv("INTENT_FILE")
intents_path = os.getenv("INTENT_PATH")
intents_file = os.path.join(intents_path, intents_name)

model_name = os.getenv("MODEL_NAME")
model_path = os.getenv("MODEL_PATH")
model_file = os.path.join(model_path, model_name)

In [3]:
# Initialize WordNetLemmatizer and download NLTK data
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Abdulmunim\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abdulmunim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Abdulmunim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# Initialize variables and read data from intents.json
words = []
classes = []
documents = []
ignore_words = ["?", "!"]
data_file = open(intents_file).read()
intents = json.loads(data_file)

In [5]:
# Process intents and extract words and classes
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        # Tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Add documents
        documents.append((w, intent["tag"]))

        # Add classes to the class list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [6]:
# Lemmatize words and create a sorted unique word list
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))


In [7]:
# Sort classes
classes = sorted(list(set(classes)))

In [8]:
# Print some statistics
print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

106 documents
34 classes ['about', 'academic_status', 'admission_committees', 'admission_fees', 'advanced_standing_admission', 'extension_distance_undergraduate_admission', 'extension_programs', 'goodbye', 'grading_system', 'graduation_fees', 'graduation_requirements', 'greetings', 'history', 'history1', 'history2', 'history3', 'history4', 'in_service_undergraduate_admission', 'introduction', 'masters_programs', 'other_fees', 'postgraduate_programs', 'readmission_transfer_fees', 'reexamination', 'regular_undergraduate_admission', 'thanks', 'tuition_fees_ethiopian_evening', 'tuition_fees_foreign_postgraduate', 'tuition_fees_foreign_undergraduate', 'undergraduate_admission_process', 'undergraduate_graduation', 'undergraduate_programs', 'unknown', 'withdrawal_readmission']
159 unique lemmatized words ["'m", "'s", ',', '.', 'a', 'aait', 'aau', 'ababa', 'about', 'academic', 'achievement', 'addis', 'additional', 'admission', 'advanced', 'afternoon', 'allow', 'am', 'an', 'and', 'application',

In [9]:
# Save words and classes to pickle files
WORDS_FILE = os.getenv("WORDS_FILE")
WORDS_PATH = os.getenv("WORDS_PATH")
WORDS = os.path.join(WORDS_PATH, WORDS_FILE)

CLASSES_FILE = os.getenv("CLASSES_FILE")
CLASSES_PATH = os.getenv("CLASSES_PATH")
CLASSES = os.path.join(CLASSES_PATH, CLASSES_FILE)

pickle.dump(words, open(WORDS, "wb"))
pickle.dump(classes, open(CLASSES, "wb"))

In [10]:
# Initialize training data
training = []
output_empty = [0] * len(classes)

In [11]:
# Process documents to create a bag of words and output labels
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [12]:
# Shuffle training data
random.shuffle(training)

In [13]:
# Separate bag-of-words representations and output labels
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]

In [14]:
# Convert to NumPy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)

In [15]:
# Create a neural network model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

# Compile the model using SGD optimizer
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               20480     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 34)                2210      
                                                                 
Total params: 30946 (120.88 KB)
Trainable params: 30946 (120.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Fit the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
22/22 [==============================] - 1s 4ms/step - loss: 3.5577 - accuracy: 0.0377
Epoch 2/200
22/22 [==============================] - 0s 1ms/step - loss: 3.5200 - accuracy: 0.0566
Epoch 3/200
22/22 [==============================] - 0s 1ms/step - loss: 3.4143 - accuracy: 0.0755
Epoch 4/200
22/22 [==============================] - 0s 1ms/step - loss: 3.3550 - accuracy: 0.1132
Epoch 5/200
22/22 [==============================] - 0s 1ms/step - loss: 3.2845 - accuracy: 0.1321
Epoch 6/200
22/22 [==============================] - 0s 1ms/step - loss: 3.1779 - accuracy: 0.1509
Epoch 7/200
22/22 [==============================] - 0s 1ms/step - loss: 3.1418 - accuracy: 0.1887
Epoch 8/200
22/22 [==============================] - 0s 1ms/step - loss: 2.9898 - accuracy: 0.2264
Epoch 9/200
22/22 [==============================] - 0s 1ms/step - loss: 2.8535 - accuracy: 0.2170
Epoch 10/200
22/22 [==============================] - 0s 1ms/step - loss: 2.7476 - accuracy: 0.2264
Epoch 11/

In [17]:
# Save the model
model.save(model_file, hist)

c:\Users\Abdulmunim\Desktop\SYNC\AAU-Chatbot\engine\.venv\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# DONE